In [ ]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:15:52tcmalloc: large alloc 1147494400 bytes == 0x55ce9da76000 @  0x7fb99d8d7615 0x55ce6417c02c 0x55ce6425c17a 0x55ce6417ee4d 0x55ce64270c0d 0x55ce641f30d8 0x55ce641edc35 0x55ce6418073a 0x55ce641f2f40 0x55ce641edc35 0x55ce6418073a 0x55ce641ef93b 0x55ce64271a56 0x55ce641eefb3 0x55ce64271a56 0x55ce641eefb3 0x55ce64271a56 0x55ce641eefb3 0x55ce64180b99 0x55ce641c3e79 0x55ce6417f7b2 0x55ce641f2e65 0x55ce641edc35 0x55ce6418073a 0x55ce641ef93b 0x55ce641edc35 0x55ce6418073a 0x55ce641eeb0e 0x55ce6418065a 0x55ce641eed67 0x55ce641edc35
     |████████████████▌               | 1055.7 MB 80.2 MB/s eta 0:00:13tcmalloc: large alloc 1434370048 bytes == 0x55cee20cc000 @  0x7fb99d8d7615 0x55ce6417c02c 0x55ce6425c17a 0x55ce6417ee4d 0x55ce64270c0d 0x55ce641f30d8 0x55ce641edc35 0x55ce6418073a 0x55ce641f2f40 0x55ce641edc35 0x55ce6418073a 0x55ce641ef93b 0x55ce64271a56 0x55ce641ee

In [ ]:
!nvidia-smi

In [ ]:
! pip install datasets transformers rouge-score nltk deepspeed
import nltk
from datasets import load_from_disk
from datasets import Dataset
import datasets
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, PretrainedConfig, AutoModel, AutoModelForPreTraining, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import os

In [ ]:
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994' # modify if RuntimeError: Address already in use
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"

In [ ]:
# !git clone https://github.com/microsoft/DeepSpeed/
# !cd DeepSpeed
# !rm -rf build
# !TORCH_CUDA_ARCH_LIST="6.0" DS_BUILD_CPU_ADAM=1 DS_BUILD_UTILS=1 pip install . \
# !--global-option="build_ext" --global-option="-j8" --no-cache -v \
# !--disable-pip-version-check 2>&1 | tee build.log

In [ ]:
!huggingface-cli login
!pip install hf-lfs
!git config --global user.email "qichang.zheng18@student.xjtlu.edu.cn"
!git config --global user.name "QichangZheng"


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: qichang.zheng18@student.xjtlu.edu.cn
Password: 
Login successful
Your token: MJiGCmOowBrkkXnCwLRcvopDGWWgfWvpVjUAkTVPaRSVgzYiRhffKAPIFwWGLqsOqSkhCOfDdtFzTnDrklzaGRBECwFfArdeROvFkneSoXKJHNBTqIYoqubMiNvjhLTV 

Your token has been saved to /root/.huggingface/token


In [ ]:
nltk.download('punkt')
model_checkpoint = 'facebook/bart-large-cnn'

In [ ]:
# raw_datasets = load_from_disk("/content/drive/MyDrive/Patent_Dataset/Test_dataset_with_validation")
raw_datasets = load_from_disk("/content/drive/MyDrive/Patent_Dataset/Small_dataset")

In [ ]:
# config = PretrainedConfig(name_or_path=model_checkpoint, min_length=0, max_length=1024, early_stopping=True, num_beams=4)#min_length=0, max_length=1024, num_beams=2, length_penalty=1, early_stopping=True
# print(type(config))
# tokenizer = BartTokenizer.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = AutoModelForPreTraining.from_pretrained(model_checkpoint)
# model = BartForConditionalGeneration.from_pretrained(model_checkpoint, min_length=0, max_length=1024, early_stopping=True)
# model = AutoModel.from_pretrained(model_checkpoint)#, min_length=0, max_length=1024, early_stopping=True
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, min_length=0, max_length=1024, early_stopping=True)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
max_input_length = 1024
max_target_length = 1024

def preprocess_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

## **Fine-tune Process**

In [ ]:
%%bash
cat <<'EOT' > ds_config_zero3.json
{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },

    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": true
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e9,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "stage3_gather_fp16_weights_on_model_save": true
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}
EOT

In [ ]:
batch_size = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/Transformers/Output',
    resume_from_checkpoint=False,
    overwrite_output_dir=True,
    eval_steps=1000000,
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    save_total_limit=3,
    save_steps=20000,
    logging_steps=50000,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=False,
    # push_to_hub_model_id=f"{model_name}-finetuned-xsum",
    deepspeed="ds_config_zero3.json"
)

[2021-08-18 18:15:45,081] [INFO] [distributed.py:47:init_distributed] Initializing torch distributed with backend: nccl


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # # Replace -100 in the labels as we can't decode them.
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # # Rouge expects a newline after each sentence
    # decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    # decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # # Extract a few results
    # result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result = np.mean(prediction_lens)
    
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
    # compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
# !nvidia-smi

In [ ]:
trainer.train()
# trainer.train(from_checkpoint=True)

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.


[2021-08-18 18:15:50,169] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed info: version=0.5.0, git-hash=unknown, git-branch=unknown
[2021-08-18 18:16:00,825] [INFO] [logging.py:68:log_dist] [Rank 0] initializing deepspeed groups
[2021-08-18 18:16:00,826] [INFO] [logging.py:68:log_dist] [Rank 0] initializing deepspeed model parallel group with size 1
[2021-08-18 18:16:00,829] [INFO] [logging.py:68:log_dist] [Rank 0] initializing deepspeed expert parallel group with size 1
[2021-08-18 18:16:00,832] [INFO] [logging.py:68:log_dist] [Rank 0] creating expert data parallel process group with ranks: [0]
[2021-08-18 18:16:00,834] [INFO] [logging.py:68:log_dist] [Rank 0] creating expert parallel process group with ranks: [0]
[2021-08-18 18:16:00,897] [INFO] [engine.py:189:__init__] DeepSpeed Flops Profiler Enabled: False
Installed CUDA version 11.0 does not match the version torch was compiled with 11.1 but since the APIs are compatible, accepting this combination
Using /root/.cache/torch_ext

***** Running training *****
  Num examples = 6
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 6


Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...
Time to load utils op: 0.007422924041748047 seconds
[2021-08-18 18:16:10,575] [INFO] [stage3.py:2735:_overflow_clean_up] [deepscale] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 65536, reducing to 65536


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6, training_loss=nan, metrics={'train_runtime': 9.58, 'train_samples_per_second': 0.626, 'train_steps_per_second': 0.626, 'total_flos': 6473952.0, 'train_loss': nan, 'epoch': 1.0})

In [ ]:
model_name="DS-config-5"
tokenizer.push_to_hub(model_name)
model.push_to_hub(model_name)

# config1 = BartConfig(name_or_path=model_name, min_length=0, max_length=1024, num_beams=2, length_penalty=1, early_stopping=True)
# config1.push_to_hub(model_name)

NameError: ignored

In [ ]:
ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt', truncation=True)

# Generate Summary
summary_ids = model.generate(inputs['input_ids'])
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

RuntimeError: ignored

In [ ]:
model_name = "Pyke/DS-config-5"
# config = BartConfig.from_pretrained(model_name)

In [ ]:
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=10, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

Downloading:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

RuntimeError: ignored